In [40]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot
import plotly.express as px
import lmfit

covid_stats=pd.read_csv("../../covid-statistics-by-us-states-daily-updates.csv")

In [41]:
covid_stats=covid_stats[covid_stats.state=='NY']
covid_stats=covid_stats.drop(['Unnamed: 0', 'posneg', 'hash', 'commercialscore', 'negativeregularscore',
                             'negativescore', 'positivescore', 'grade', 'score','state','dataqualitygrade',
                              'lastupdateet', 'datemodified','checktimeet', 'datechecked', 'fips', 'hash',
                              'commercialscore','negativeregularscore', 'negativescore', 'positivescore',
                              'score', 'grade'], 
                             axis=1)
covid_stats.head()

,date,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,onventilatorcumulative,...,positivecasesviral,deathconfirmed,deathprobable,positiveincrease,negativeincrease,total,totaltestresults,totaltestresultsincrease,deathincrease,hospitalizedincrease
37,2020-07-29,413593.0,5333229.0,NaN,619.0,89995.0,154.0,NaN,76.0,NaN,...,413593.0,NaN,NaN,715,61561,5746822,5746822,62276,6,0
93,2020-07-28,412878.0,5271668.0,NaN,648.0,89995.0,152.0,NaN,81.0,NaN,...,412878.0,NaN,NaN,534,56863,5684546,5684546,57397,9,0
149,2020-07-27,412344.0,5214805.0,NaN,642.0,89995.0,149.0,NaN,84.0,NaN,...,412344.0,NaN,NaN,608,56662,5627149,5627149,57270,11,0
205,2020-07-26,411736.0,5158143.0,NaN,637.0,89995.0,155.0,NaN,90.0,NaN,...,411736.0,NaN,NaN,536,53032,5569879,5569879,53568,3,0
261,2020-07-25,411200.0,5105111.0,NaN,646.0,89995.0,149.0,NaN,94.0,NaN,...,411200.0,NaN,NaN,750,70716,5516311,5516311,71466,13,0


In [42]:
covid_stats = covid_stats.sort_values('date')

In [43]:
data = covid_stats.filter(['date','positiveincrease'], axis=1)
fig = px.line(x=data['date'],
              y=data['positiveincrease'],
              title='positive increase')
fig.show()
data = data['positiveincrease'].values

In [44]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

# The SIR model differential equations.
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

In [51]:
def Model(beta, gamma):
    # Total population, N.
    N = 19453556
    # Initial number of infected and recovered individuals, I0 and R0.
    I0, R0 = 1, 0
    # Everyone else, S0, is susceptible to infection initially.
    S0 = N - I0 - R0
    # Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
    t = np.linspace(0, 148, 148,dtype=int)
    y0 = S0, I0, R0
    ret = odeint(deriv, y0, t, N, args = (beta,gamma))
    S, I, R = ret.T
    return t, S, I, R

In [49]:
def fitter(x, beta, gamma):
    ret = Model(beta,gamma)
    return ret[3][x]

In [52]:
mod = lmfit.Model(fitter)
mod.set_param_hint("beta", value = 0.7, vary = True)
mod.set_param_hint("gamma", value = 0.2 , vary = True)
params = mod.make_params()
t = np.linspace(0, 148, 148,dtype=int)
result = mod.fit(data, params, method="least_squares", x=t)
# result

TypeError: odeint() got multiple values for argument 'args'